In [1]:
from yahoo_fin import options
import numpy as np
from yahoo_fin import options
import yfinance as yf

ticker = 'intc'

ticker_yf = yf.Ticker(ticker)

ticker_yf.options
exparr = np.asarray(ticker_yf.options)
exparr

array(['2023-01-13', '2023-01-20', '2023-01-27', '2023-02-03',
       '2023-02-10', '2023-02-17', '2023-02-24', '2023-03-17',
       '2023-04-21', '2023-06-16', '2023-07-21', '2023-09-15',
       '2024-01-19', '2025-01-17'], dtype='<U10')

In [2]:
#Creo array con tutte le opzioni [expiration date] [strike] [feature]
all_arr=[None] * exparr.size #array che contiene arrays, ogni arrays ha al suo interno tutte le opzioni con una determinata scadenza
for i in range(exparr.size):
    opt=np.asarray(ticker_yf.option_chain(date=exparr[i]).calls) #solo calls
    all_arr[i]=opt
#print(all_arr[0].size)
#print(exparr.size)

int1=[0] * exparr.size
int2=[0] * exparr.size
for i in range(exparr.size):
    int1[i],int2[i]=all_arr[i].shape
    print(int1[i])



32
60
24
20
16
27
13
18
28
36
25
9
25
14


In [3]:
import pandas as pd
data = ({
    'FirstContract':["x"],
    'SecondContract':["y"],
    'FirstStrike':[0],
    'SecondStrike':[0],
    'FirstAskPremium':[0],
    'SecondBidPremium':[0],
    'RendimentoPerc' :[0],
    'RischioDollar':[0]
               })
df = pd.DataFrame(data)
print(df)

  FirstContract SecondContract  FirstStrike  SecondStrike  FirstAskPremium  \
0             x              y            0             0                0   

   SecondBidPremium  RendimentoPerc  RischioDollar  
0                 0               0              0  


In [4]:
#GET ALL PAIRS OF ACTIONS
for i in range(exparr.size):
    for j in range(int1[i]):
        for k in range(int1[i]):
            #print(k)
            rendimento = all_arr[i][j][5]-all_arr[i][k][5]
            firstContracts = all_arr[i][j][0]
            secondContracts = all_arr[i][k][0]
            firstStrike = all_arr[i][j][2]
            secondStrike = all_arr[i][k][2]
            firstAskPremium = all_arr[i][j][5]
            secondBidPremium = all_arr[i][k][4]
            netpremium = firstAskPremium-secondBidPremium
            if (netpremium != 0):
                rendimento = ((secondStrike-firstStrike-netpremium)/netpremium)*100
            else:
                rendimento = 0
            #if(firstStrike<secondStrike):
            new_row = {'FirstContract': firstContracts, 'SecondContract': secondContracts,
            'FirstStrike': firstStrike, 'SecondStrike': secondStrike,
            'FirstAskPremium': firstAskPremium, 'SecondBidPremium': secondBidPremium,
            'RendimentoPerc': rendimento, 'RischioDollar': (netpremium*100)}
            df = df.append(new_row, ignore_index=True)

In [5]:
#QUERY
dfQuery = df.sort_values('RendimentoPerc',ascending=False).query('FirstStrike<SecondStrike').query('RendimentoPerc>300').query('RischioDollar<400').query('RischioDollar>100').query('SecondStrike-FirstStrike<10')
print(dfQuery)
dfQuery.to_csv('/Users/alessandro/Desktop/MenthorQPyForFin/'+ticker+'.csv')

             FirstContract       SecondContract  FirstStrike  SecondStrike  \
4959   INTC230127C00030000  INTC230127C00039000         30.0          39.0   
9558   INTC230721C00035000  INTC230721C00044000         35.0          44.0   
9557   INTC230721C00035000  INTC230721C00043000         35.0          43.0   
4958   INTC230127C00030000  INTC230127C00038000         30.0          38.0   
5399   INTC230203C00030000  INTC230203C00039000         30.0          39.0   
...                    ...                  ...          ...           ...   
5727   INTC230210C00029000  INTC230210C00036000         29.0          36.0   
4909   INTC230127C00028000  INTC230127C00037000         28.0          37.0   
10101  INTC240119C00035000  INTC240119C00040000         35.0          40.0   
9478   INTC230721C00032000  INTC230721C00039000         32.0          39.0   
8388   INTC230616C00030000  INTC230616C00039000         30.0          39.0   

       FirstAskPremium  SecondBidPremium  RendimentoPerc  Risch